In [1]:
import json
with open("./small_dataset.json") as f:
    dic = json.load(f)
    dic = dic["data"] # when run Launcher.ipynb, comment this out

# Parse data into assets and
from utils import json_parse
price_df, esg_s, asset_arr = json_parse(dic)

# Calculates mu and sigma using OAS, LedoitWolf or normal cov
from utils import compute_mu_sigma
mu, Sigma = compute_mu_sigma(price_df, lookback=252, shrinkage=True)

/home/jovyan/work/git/singapore25_challenge3_team9/utils/Pre_processing.py:28: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices_df.pct_change().dropna() # percentage change between the current element and prior element


In [2]:
from utils import PortfolioOptimization, print_result, run_QAOA
from quantinuum_wrapper import QuantinuumWrapper
backend=QuantinuumWrapper.get_target()

# Parameters
q = 1  
num_assets = len(mu)
budget = num_assets // 2  
portfolio = PortfolioOptimization(expected_returns=mu, covariances=Sigma, risk_factor=q, budget=budget, esg_scores=esg_s, roi_factor=1, esg_factor=0.01)
qp = portfolio.to_quadratic_program()
# Code for backend
result = run_QAOA(qp, backend)

q = print_result(result, portfolio)

Optimal: selection [1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0.], value -1.9661

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 1 0 0 1 1 0 0 1 0 0 1 1]	0.2641		0.0098
[0 0 1 1 0 1 0 1 0 1 1 1 0]	0.5626		0.0088
[0 1 0 0 1 1 1 0 1 0 0 0 1]	-0.7330		0.0078
[0 1 0 0 1 1 0 1 1 0 0 1 1]	0.5830		0.0078
[1 0 1 1 0 0 0 1 0 1 1 1 0]	-0.0869		0.0078
[1 0 1 1 1 0 1 0 0 1 1 0 0]	-1.7082		0.0078
[1 0 1 1 0 0 1 1 0 1 1 0 0]	-1.1043		0.0068
[0 1 0 0 1 1 0 0 1 0 0 0 1]	-0.4179		0.0068
[0 0 0 1 0 0 0 1 0 1 1 1 0]	0.4139		0.0068
[1 0 1 1 0 0 1 0 0 1 1 1 0]	-0.6594		0.0059
[1 0 1 1 0 0 1 1 0 1 1 1 0]	-0.3539		0.0059
[1 1 0 0 1 1 1 0 1 0 0 0 1]	-0.9455		0.0059
[0 0 0 1 0 0 0 1 0 0 1 1 0]	0.6000		0.0049
[0 0 0 1 0 0 0 1 0 1 0 1 0]	0.6329		0.0049
[1 1 1 1 0 0 1 1 0 1 1 0 0]	-1.0937		0.0049
[0 1 0 0 1 1 0 1 0 0 0 1 1]	0.5074		0.0049
[0 1 0 0 0 1 0 1 1 0 0 1 1]	0.9362		0.0049
[1 0 0 1 0 0 0 1 0 0 0 1 0]	0.5277		0.0049
[1 0 1 0

In [5]:
print(q)

[1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0.]


In [10]:
%%timeit
result = qaoa.solve(qp)

6.16 s ± 474 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
